<a href="https://colab.research.google.com/github/wel51x/DS-Unit-2-Sprint-5-Predictive-Modeling-Challenge/blob/master/DS2_Unit2_Sprint5_Day_3_Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 2 Sprint 5 DS2 Predictive Modeling Challenge

## Day 3 - RandomForestClassifier

In [0]:
# for colab must re-upload files after running this
!pip install category_encoders

    100% |████████████████████████████████| 61kB 2.4MB/s 


In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
import warnings
import time

start_time = time.time()
pd.set_option('display.max_columns', None) # all cols
pd.set_option('display.width', 161)
warnings.filterwarnings('ignore')

# get data
#path = "https://github.com/wel51x/DS-Unit-2-Sprint-5-Predictive-Modeling-Challenge/tree/master/data/"
train_features_file = "train_features.csv"
train_labels_file = "train_labels.csv"
test_features_file = "test_features.csv"
submission_in_file = "sample_submission.csv"

train_features = pd.read_csv(train_features_file)
train_labels = pd.read_csv(train_labels_file)
test_features = pd.read_csv(test_features_file)

print("\ntrain_features null counts:")
print(train_features.isnull().sum())
print(train_labels.status_group.value_counts())
print("\ntest_features null counts:")
print(test_features.isnull().sum())
# dropna
train_features.dropna(axis=1, inplace=True)
test_features.dropna(axis=1, inplace=True)
print(train_features.isnull().sum().sum())
print(test_features.isnull().sum().sum())

# create age feature
train_features['newer'] = (train_features.construction_year > 1998).map({True : 1, False : 0})
test_features['newer'] = (test_features.construction_year > 1998).map({True : 1, False : 0})

print("\ntrain_features unique nonumeric values")
print(train_features.select_dtypes(exclude=np.number).nunique())

y_train = train_labels.status_group
# drop id + "high cardinality"
X_train = train_features.drop(['id', "date_recorded", "wpt_name", "ward"], axis = 1)
X_test  = test_features.drop(['id', "date_recorded", "wpt_name", "ward"], axis = 1)

cat_encoder = ce.OneHotEncoder(use_cat_names=True)

clf = RandomForestClassifier(class_weight = 'balanced',
                                            n_jobs = -1,
                                            n_estimators = 2000)
pipeline = make_pipeline(cat_encoder,
                         StandardScaler(),
                         clf)

pipe_out = pipeline.fit(X_train, y_train)

print("RF Score:", pipeline.score(X_train, y_train))

#get predictions & stick in submission
submission = pd.read_csv(submission_in_file)

temp = pd.DataFrame(pipeline.predict(X_test))

print(temp.shape)
print(temp[0].value_counts(normalize=True))
print(temp[0].value_counts(normalize=True))

submission['status_group'] = temp[0]
print(submission.status_group.value_counts())

submission_out_file = "submission-day3.csv"
submission.to_csv(submission_out_file, index=False)
print("--- Run time: %s seconds ---" % (time.time() - start_time))
# Best score so far: 0.81111


train_features null counts:
id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_qual